In [9]:
# ============================================
# CAPTURA DE WEBCAM + DESCRIPCIÓN GPT-4 VISION
# Google Colab Notebook
# ============================================

# PASO 1: Instalar dependencias
print("Instalando dependencias...")
!pip install -q openai pillow


Instalando dependencias...


In [10]:
# PASO 2: Importar librerías
import os
import base64
from io import BytesIO
from IPython.display import display, Javascript, Image as IPImage
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
import openai

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


In [ ]:
# ============================================
# CONFIGURACIÓN DE API KEY
# ============================================

# Ingresa tu API Key de OpenAI
API_KEY = ""  # ⚠️ IMPORTANTE: Ingresa tu API key aquí

if not API_KEY:
    print("INGRESA TU API KEY DE OPENAI")
    print(" Obtén tu API key en: https://platform.openai.com/api-keys")
else:
    client = openai.OpenAI(api_key=API_KEY)
    print("Cliente de OpenAI configurado")


Cliente de OpenAI configurado


In [12]:
# ============================================
# FUNCIÓN PARA CAPTURAR IMAGEN DE WEBCAM
# ============================================

def capture_webcam_image():
    """
    Captura una imagen desde la webcam del navegador
    Retorna la imagen en formato PIL
    """
    js_code = """
    async function captureImage() {
        const div = document.createElement('div');
        const video = document.createElement('video');
        const canvas = document.createElement('canvas');

        div.style.display = 'flex';
        div.style.flexDirection = 'column';
        div.style.alignItems = 'center';
        div.style.gap = '10px';

        const captureBtn = document.createElement('button');
        captureBtn.textContent = '📸 Capturar Imagen';
        captureBtn.style.padding = '10px 20px';
        captureBtn.style.fontSize = '16px';
        captureBtn.style.cursor = 'pointer';
        captureBtn.style.backgroundColor = '#4CAF50';
        captureBtn.style.color = 'white';
        captureBtn.style.border = 'none';
        captureBtn.style.borderRadius = '5px';

        div.appendChild(video);
        div.appendChild(captureBtn);
        document.body.appendChild(div);

        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        video.srcObject = stream;
        await video.play();

        // Esperar a que el video esté listo
        await new Promise(resolve => {
            video.onloadedmetadata = resolve;
        });

        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;

        // Esperar el click del botón
        await new Promise(resolve => {
            captureBtn.onclick = resolve;
        });

        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getTracks().forEach(track => track.stop());
        div.remove();

        return canvas.toDataURL('image/jpeg', 0.9);
    }
    """

    print("📷 Abriendo webcam... Haz clic en 'Capturar Imagen' cuando estés listo")
    data = eval_js('captureImage()')

    # Decodificar la imagen
    image_data = data.split(',')[1]
    binary_data = b64decode(image_data)
    image = Image.open(BytesIO(binary_data))

    print("✅ Imagen capturada exitosamente")
    return image

In [13]:
# ============================================
# FUNCIÓN PARA DESCRIBIR IMAGEN CON GPT-4
# ============================================

def describe_image_with_gpt4(image, prompt="Describe esta imagen en detalle"):
    """
    Envía la imagen a GPT-4 Vision para obtener una descripción
    """
    # Convertir imagen a base64
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    img_base64 = base64.b64encode(buffered.getvalue()).decode()

    print("🤖 Enviando imagen a GPT-4 Vision...")

    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # o "gpt-4-vision-preview" si prefieres
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{img_base64}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=500
        )

        description = response.choices[0].message.content
        return description

    except Exception as e:
        return f"❌ Error al procesar la imagen: {str(e)}"

In [14]:
# ============================================
# FUNCIÓN PRINCIPAL
# ============================================

def run_webcam_gpt4():
    """
    Función principal que ejecuta todo el proceso
    """
    print("\n" + "="*50)
    print("🎥 CAPTURA DE WEBCAM + GPT-4 VISION")
    print("="*50 + "\n")

    if not API_KEY:
        print("⚠️  ERROR: No se ha configurado la API Key de OpenAI")
        print("Por favor, edita la variable API_KEY al inicio del código")
        return

    # Capturar imagen
    image = capture_webcam_image()

    # Mostrar la imagen capturada
    print("\n📸 Imagen capturada:")
    display(image)

    # Obtener descripción de GPT-4
    print("\n" + "-"*50)
    description = describe_image_with_gpt4(image)

    print("\n🤖 DESCRIPCIÓN DE GPT-4:")
    print("-"*50)
    print(description)
    print("-"*50)

    return image, description

# ============================================
# EJECUTAR EL PROGRAMA
# ============================================

# Ejecuta esta función para iniciar la captura
# run_webcam_gpt4()

print("\n" + "="*50)
print("✨ Todo listo!")
print("="*50)
print("\n📝 INSTRUCCIONES:")
print("1. Ingresa tu API Key de OpenAI en la variable API_KEY")
print("2. Ejecuta: run_webcam_gpt4()")
print("3. Permite el acceso a la cámara cuando se solicite")
print("4. Haz clic en 'Capturar Imagen' cuando estés listo")
print("5. Espera la descripción de GPT-4")
print("\n🚀 Ejemplo: run_webcam_gpt4()")
print("="*50)


✨ Todo listo!

📝 INSTRUCCIONES:
1. Ingresa tu API Key de OpenAI en la variable API_KEY
2. Ejecuta: run_webcam_gpt4()
3. Permite el acceso a la cámara cuando se solicite
4. Haz clic en 'Capturar Imagen' cuando estés listo
5. Espera la descripción de GPT-4

🚀 Ejemplo: run_webcam_gpt4()
